# The scalar advection equation

In this notebook, we'll show how to use the solvers in this package on a relatively simple problem, the linear conservative advection equation.
First, we'll import the Firedrake package, create the geometry, and then create a function space.
The geometry will be the unit square in 2D and, as a first pass, we'll use piecewise constant functions to represent our solution.

In [ ]:
import firedrake
nx, ny = 32, 32
mesh = firedrake.PeriodicUnitSquareMesh(nx, ny, diagonal='crossed')
Q0 = firedrake.FunctionSpace(mesh, 'DG', 0)

Next, we'll create the velocity field, which will be uniform solid body rotation about the center of the domain.
The function `firedrake.SpatialCoordinate` returns an object `x` that represents symbolically an arbitrary point of the domain.
We can then form symbolic expressions that represent functions of space by manipulating this object `x` algebraically.

In [ ]:
from firedrake import as_vector, Constant
x = firedrake.SpatialCoordinate(mesh)
y = Constant((0.5, 0.5))
w = x - y
u = as_vector((-w[1], +w[0]))

We won't include any sources or sinks for now.
We're creating this variable `s` to represent sources and sinks because we'll need to pass that to the function that forms the advection equation for us.

In [ ]:
s = Constant(0.)

Now we'll create the initial value of the state variable.
Once again, we'll create an expression through algebraic manipulation of the spatial coordinates `x` of the domain.

In [ ]:
from firedrake import inner, max_value
z = Constant((1 / 3, 1 / 3))
r = Constant(1 / 6)
expr = max_value(0, 1 - inner(x - z, x - z) / r**2)

This expression object is purely symbolic -- it doesn't have an array of coefficients because it hasn't been discretized yet.
To create a discretized function, we'll `project` that expression into the function space that we created earlier.

In [ ]:
q_0 = firedrake.project(expr, Q0)

Firedrake has some built-in hooks to matplotlib, so we can use this to check and make sure that we're actually prescribing the right initial condition.

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q_0, axes=axes)
fig.colorbar(colors);

In order to get one full rotation, we'll use a final time of $2\pi$.
We then need to pick a timestep that will satisfy the Courant-Friedrichs-Lewy condition, since we'll be using an explicit timestepping scheme.

In [ ]:
import numpy as np
final_time = 2 * np.pi
min_diameter = mesh.cell_sizes.dat.data_ro[:].min()
print(f'Smallest cell diameter: {min_diameter}')
max_speed = 1 / np.sqrt(2)
timestep = (min_diameter / 8) / max_speed
num_steps = int(final_time / timestep)
dt = final_time / num_steps

Now we're getting to the good part.
The subpackage `plumes.models` includes several Python modules.
Each of those modules defines a different kind of model.
Here we're looking at the advection equation because it's simple and it's a good test problem for debugging numerics and trying out different spatial or temporal discretization schemes.
We'll look at other models and finally the plume model later.

A physics model is described entirely by whatever the right-hand side of the evolution equation is.
For the advection model, the evolution equation is

$$\partial_tq = -\nabla\cdot (qu) + s.$$

The function `make_equation` that lives in the advection module takes in the velocity field $u$ and the source terms $s$.
It returns another function that will then calculate the discretized form of the right-hand side of the last equation.

In [ ]:
from plumes import models
equation = models.advection.make_equation(u, s)

To solve this equation numerically, we'll create an integrator object, which lives in the module `plumes.numerics`.
Here we'll use the very simple explicit Euler scheme.
The integrator takes in the equation to be solve, the initial state, and a starting timestep.

In [ ]:
from plumes import numerics
integrator = numerics.ExplicitEuler(equation, q_0)

The integrator object has one job: to step the model forward by a given timestep.
For this demo, we're passing the same value of the timestep on every iteration.
Later we'll show how to do adaptive timestepping.

The current solution is stored in the member `integrator.state`.
What we'd like to do is extract the value of the solution every 30th of a second.
We've also used the package `tqdm` to add a progress bar.

In [ ]:
import tqdm

output_time = 1/30
output_freq = int(output_time / dt)

qs = []
for step in tqdm.trange(num_steps):
    q = integrator.state
    if step % output_freq == 0:
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

Now we can make a movie to visualize the results.

In [ ]:
%%capture
fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
axes.set_xlim((0, 1))
axes.set_ylim((0, 1))
colors = firedrake.tripcolor(
    q, num_sample_points=1, vmin=0., vmax=1., axes=axes
)

from matplotlib.animation import FuncAnimation
def animate(q):
    colors.set_array(q.dat.data_ro[:])

interval = 1e3 * output_freq * dt
animation = FuncAnimation(fig, animate, frames=qs, interval=interval)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

The solution has become very diffused compared to its original value.

In [ ]:
from firedrake import assemble, dx
assemble(abs(q - expr) * dx) / assemble(abs(expr) * dx)

Nonetheless, the total volume is conserved.

In [ ]:
abs(assemble((q - expr) * dx))

Moreover, the solution remains positive.
Many high-order accurate discretization schemes are not positivity preserving.

In [ ]:
q.dat.data_ro[:].min()